In [34]:
from sympy import init_session
import math
import matplotlib.pyplot as plt
import sys
import ipywidgets as widgets
import numpy as np
import imageio.v2 as imageio
from IPython.display import Image
import pandas as pd
import seaborn as sns

In [35]:

m = 3.48 #kg
l = 24.5 * 25.4 #length of wiper mm
I = 1/12 * m * l**2 #moment of inertia about com kgm^2

In [69]:
#Parameters
motor_torque = 20 #nM 
df = pd.read_csv("4bardata.txt", sep='\t' , header = None, names = ["theta_2 [rad]", "theta_3 [rad]", "theta_4 [rad]", "theta_2 [°]","theta_3 [°]","theta_4 [°]","omega_2 [rad/s]", "omega_43 [rad/s]","omega_4 [rad/s]","alpha_2 [rad/s^2]","alpha_3 [rad/s^2]","alpha_4 [rad/s^2]"])

df.head()
print(df.shape[0])

301
